In [1]:
import numpy as np
import pandas as pd

In [2]:
zipcodes = pd.read_csv('Seattle_Building_Energy_Benchmarking_2016_raw.csv',  index_col='OSEBuildingID', usecols=['OSEBuildingID','ZipCode'])
data_2015 = pd.read_csv('Seattle_Building_Energy_Benchmarking_2015_raw.csv')
data_2016 = pd.read_csv('Seattle_Building_Energy_Benchmarking_2016_raw.csv')
data_2017 = pd.read_csv('Seattle_Building_Energy_Benchmarking_2017_raw.csv')

### Functions for cleaning data

In [3]:
def fill_propertytype(df):
    for i in range(len(df)):
        if pd.isnull(df['LargestPropertyUseType'][i]):
            df.at[i, 'LargestPropertyUseType'] = df['PrimaryPropertyType'][i]
        else:
            continue         

def reclassify_buildingtype(df):
    residential = ['Multifamily LR (1-4)', 'Multifamily MR (5-9)', 'Multifamily HR (10+)']
    specialty = ['Campus', 'SPS-District K-12', 'Nonresidential COS']
    for i in range(len(df)):
        if df['BuildingType'][i] in residential:
            df.at[i, 'BuildingType'] = 'Residential'
        elif df['BuildingType'][i] in specialty:
            df.at[i, 'BuildingType'] = 'Specialty'
        else:
            continue           

def remove_outliers(df):   
    for i in range(len(df)):
        if pd.isnull(df['Outlier'][i]):
            continue 
        else:
            df.drop([i], inplace=True)
    df.reset_index(inplace=True)
            
def remove_empty_rows(df):   
    for i in range(len(df)):
        if pd.isnull(df['ZipCode'][i]):
             df.drop([i], inplace=True)
        else:
            continue
    df.reset_index(inplace=True)
            
def remove_columns_Seattle_2015(df):
    df.drop(['PrimaryPropertyType',
             'PropertyName',
             'TaxParcelIdentificationNumber', 
             'Location',
             'CouncilDistrictCode',
             'Neighborhood',
             'DefaultData',
             'Comment',
             'ComplianceStatus',
             'Outlier',
             '2010 Census Tracts',
             'Seattle Police Department Micro Community Policing Plan Areas',
             'SPD Beats',
             'NumberofBuildings',
             'NumberofFloors',
             'PropertyGFAParking',
             'PropertyGFABuilding(s)',
             'ListOfAllPropertyUseTypes',
             'LargestPropertyUseTypeGFA',
             'SecondLargestPropertyUseType',
             'SecondLargestPropertyUseTypeGFA',
             'ThirdLargestPropertyUseType',
             'ThirdLargestPropertyUseTypeGFA',
             'YearsENERGYSTARCertified',
             'ENERGYSTARScore',
             'GHGEmissions(MetricTonsCO2e)',
             'GHGEmissionsIntensity(kgCO2e/ft2)',
             'City Council Districts',
             'Electricity(kWh)',
             'NaturalGas(therms)',
             'Zip Codes',
             'OtherFuelUse(kBtu)'
           ], inplace=True, axis=1)            

def remove_columns_Seattle_2016(df):
    df.drop(['PrimaryPropertyType',
             'PropertyName',
             'Address',
             'TaxParcelIdentificationNumber', 
             'CouncilDistrictCode',
             'Neighborhood',
             'Latitude',
             'Longitude',
             'DefaultData',
             'Comments',
             'ComplianceStatus',
             'Outlier',
             'NumberofBuildings',
             'NumberofFloors',
             'PropertyGFAParking',
             'PropertyGFABuilding(s)',
             'ListOfAllPropertyUseTypes',
             'LargestPropertyUseTypeGFA',
             'SecondLargestPropertyUseType',
             'SecondLargestPropertyUseTypeGFA',
             'ThirdLargestPropertyUseType',
             'ThirdLargestPropertyUseTypeGFA',
             'YearsENERGYSTARCertified',
             'ENERGYSTARScore',
             'TotalGHGEmissions',
             'GHGEmissionsIntensity',
             'Electricity(kWh)',
             'NaturalGas(therms)',
           ], inplace=True, axis=1)

def remove_columns_Seattle_2017(df):
    df.drop(['PrimaryPropertyType',
             'PropertyName',
             'Address',
             'TaxParcelIdentificationNumber', 
             'CouncilDistrictCode',
             'Neighborhood',
             'Latitude',
             'Longitude',
             'DefaultData',
             'ComplianceStatus',
             'Outlier',
             'NumberofBuildings',
             'NumberofFloors',
             'PropertyGFAParking',
             'PropertyGFABuilding(s)',
             'ListOfAllPropertyUseTypes',
             'LargestPropertyUseTypeGFA',
             'SecondLargestPropertyUseType',
             'SecondLargestPropertyUseTypeGFA',
             'ThirdLargestPropertyUseType',
             'ThirdLargestPropertyUseTypeGFA',
             'YearsENERGYSTARCertified',
             'ENERGYSTARScore',
             'TotalGHGEmissions',
             'GHGEmissionsIntensity',
             'Electricity(kWh)',
             'NaturalGas(therms)',
           ], inplace=True, axis=1)

def remove_commas(df):
    column_names_1 = np.array(df.columns[7:])
    
    if 'BuildingID' in df.columns:
        column_names_1 = np.append(column_names_1, ['BuildingID'])
    
    for i in range(len(column_names_1)):
        column = column_names_1[i]
        df[column] = pd.to_numeric(df[column].astype(str).str.replace(',',''), errors='coerce')
        
def convert_to_int(df):
    if 'BuildingID' in df.columns:
        column_names_2 = ['BuildingID', 'DataYear', 'ZipCode', 'YearBuilt']
    else:
        column_names_2 = ['DataYear', 'ZipCode', 'YearBuilt']
    
    for i in range(len(column_names_2)):
        column = column_names_2[i]
        df[column] = df[column].astype('int64')

def reset_zipcodes(df):
    df.set_index('OSEBuildingID', inplace=True)
    df = pd.concat([zipcodes, data_2015], sort=True, axis=1)
    df.reset_index(inplace=True)
    
    return df

def restructure_columns(df):
    df['City'] = 'Seattle'
    df['State'] = 'WA'
    df.rename({'LargestPropertyUseType':'PrimaryPropertyUse'}, axis='columns', inplace=True)
    df = df[['OSEBuildingID',
             'DataYear',
             'City', 
             'State',
             'ZipCode',
             'BuildingType',
             'PrimaryPropertyUse',
             'YearBuilt',
             'PropertyGFATotal',
             'SiteEUI(kBtu/sf)',
             'SiteEUIWN(kBtu/sf)',
             'SourceEUI(kBtu/sf)',
             'SourceEUIWN(kBtu/sf)',
             'SiteEnergyUse(kBtu)',
             'SiteEnergyUseWN(kBtu)',
             'SteamUse(kBtu)',
             'Electricity(kBtu)',
             'NaturalGas(kBtu)']]
    
    return df

def reset_index(df):
    df.rename(index=str, columns={'OSEBuildingID':'BuildingID'}, inplace=True)
    df.set_index('BuildingID', inplace=True)

def clean_data_Seattle_2015(df): 
    fill_propertytype(df)
    reclassify_buildingtype(df)
    remove_outliers(df)
    remove_columns_Seattle_2015(df)
    df = reset_zipcodes(df)
    df.dropna(inplace=True)
    df = restructure_columns(df)
    remove_commas(df)
    convert_to_int(df)
    reset_index(df)
    return df 
    
def clean_data_Seattle_2016(df): 
    fill_propertytype(df)
    reclassify_buildingtype(df)
    remove_outliers(df)
    remove_columns_Seattle_2016(df)
    df.dropna(inplace=True)
    df = restructure_columns(df)
    remove_commas(df)
    convert_to_int(df)
    reset_index(df)
    return df
    
    
def clean_data_Seattle_2017(df): 
    fill_propertytype(df)
    reclassify_buildingtype(df)
    remove_outliers(df)
    remove_columns_Seattle_2017(df)
    remove_empty_rows(df)
    df.fillna(0, inplace=True)
    df = restructure_columns(df)
    remove_commas(df)
    convert_to_int(df)
    reset_index(df)
    return df


### Cleaning and exporting data

In [4]:
data_2015 = clean_data_Seattle_2015(data_2015)
data_2015.head()

,DataYear,City,State,ZipCode,BuildingType,PrimaryPropertyUse,YearBuilt,PropertyGFATotal,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu)
BuildingID,,,,,,,,,,,,,,,,,
1,2015,Seattle,WA,98101,NonResidential,Hotel,1927,88434,78.9,80.3,173.5,175.1,6981428,7097539,2023032,3686160,1272388
2,2015,Seattle,WA,98101,NonResidential,Hotel,1996,103566,94.4,99.0,191.3,195.2,8354235,8765788,0,3905411,4448985
3,2015,Seattle,WA,98101,NonResidential,Hotel,1969,961990,96.6,99.7,242.7,246.5,73130656,75506272,19660404,49762435,3709900
8,2015,Seattle,WA,98121,NonResidential,Hotel,1980,119890,120.1,122.1,228.8,227.1,14829099,15078243,0,6066245,8763105
9,2015,Seattle,WA,98101,Specialty,Police Station,1999,97288,135.7,146.9,313.5,321.6,12051984,13045258,0,7271004,4781283


In [5]:
data_2016 = clean_data_Seattle_2016(data_2016)
data_2016.head()

C:\Users\cjros\Anaconda3\lib\site-packages\ipykernel_launcher.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cjros\Anaconda3\lib\site-packages\ipykernel_launcher.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cjros\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).re

,DataYear,City,State,ZipCode,BuildingType,PrimaryPropertyUse,YearBuilt,PropertyGFATotal,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu)
BuildingID,,,,,,,,,,,,,,,,,
1,2016,Seattle,WA,98101,NonResidential,Hotel,1927,88434,81.699997,84.300003,182.500000,189.000000,7226362.5,7456910.0,2003882.00,3946027,1276453
2,2016,Seattle,WA,98101,NonResidential,Hotel,1996,103566,94.800003,97.900002,176.100006,179.399994,8387933.0,8664479.0,0.00,3242851,5145082
3,2016,Seattle,WA,98101,NonResidential,Hotel,1969,956110,96.000000,97.699997,241.899994,244.100006,72587024.0,73937112.0,21566554.00,49526664,1493800
5,2016,Seattle,WA,98101,NonResidential,Hotel,1926,61320,110.800003,113.300003,216.199997,224.000000,6794584.0,6946800.5,2214446.25,2768924,1811213
8,2016,Seattle,WA,98121,NonResidential,Hotel,1980,175580,114.800003,118.699997,211.399994,215.600006,14172606.0,14656503.0,0.00,5368607,8803998


In [6]:
data_2017 = clean_data_Seattle_2017(data_2017)
data_2017.head()

C:\Users\cjros\Anaconda3\lib\site-packages\ipykernel_launcher.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cjros\Anaconda3\lib\site-packages\ipykernel_launcher.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,DataYear,City,State,ZipCode,BuildingType,PrimaryPropertyUse,YearBuilt,PropertyGFATotal,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu)
BuildingID,,,,,,,,,,,,,,,,,
1,2017,Seattle,WA,98101,NonResidential,Hotel,1927,88434,83.199997,82.300003,184.399994,181.600006,7361655.0,7274452.5,2122835.75,3.950356e+06,1.288463e+06
2,2017,Seattle,WA,98101,NonResidential,Hotel,1996,103566,88.199997,86.800003,163.800003,159.800003,7804844.5,7678810.5,0.00,3.016757e+06,4.788087e+06
3,2017,Seattle,WA,98101,Specialty,Hotel,1969,956110,98.400002,98.199997,242.899994,242.600006,74470328.0,74311368.0,24313482.00,4.871284e+07,1.444000e+06
5,2017,Seattle,WA,98101,NonResidential,Hotel,1926,61320,120.199997,119.000000,234.300003,230.300003,7372221.5,7294312.0,2228119.50,3.008514e+06,2.135588e+06
8,2017,Seattle,WA,98121,NonResidential,Hotel,1980,175580,116.099998,114.099998,209.899994,205.500000,14335778.0,14081251.0,0.00,5.198202e+06,9.137576e+06


In [7]:
data_2015.to_csv(r'C:\Users\cjros\DIRECT\Capstone\Data\EnergyConsumption\Seattle\Seattle_Energy_Consumption_2015_cleaned.csv')
data_2016.to_csv(r'C:\Users\cjros\DIRECT\Capstone\Data\EnergyConsumption\Seattle\Seattle_Energy_Consumption_2016_cleaned.csv')
data_2017.to_csv(r'C:\Users\cjros\DIRECT\Capstone\Data\EnergyConsumption\Seattle\Seattle_Energy_Consumption_2017_cleaned.csv')

### Code to combine all Seattle building energy data into one .csv file

In [8]:
data_combined = pd.concat([data_2015,data_2016,data_2017], sort=False, ignore_index=False)
data_combined.reset_index(inplace=True)
data_combined

,BuildingID,DataYear,City,State,ZipCode,BuildingType,PrimaryPropertyUse,YearBuilt,PropertyGFATotal,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu)
0,1,2015,Seattle,WA,98101,NonResidential,Hotel,1927,88434,78.900000,80.300000,173.500000,175.100000,6.981428e+06,7.097539e+06,2023032.0,3.686160e+06,1.272388e+06
1,2,2015,Seattle,WA,98101,NonResidential,Hotel,1996,103566,94.400000,99.000000,191.300000,195.200000,8.354235e+06,8.765788e+06,0.0,3.905411e+06,4.448985e+06
2,3,2015,Seattle,WA,98101,NonResidential,Hotel,1969,961990,96.600000,99.700000,242.700000,246.500000,7.313066e+07,7.550627e+07,19660404.0,4.976244e+07,3.709900e+06
3,8,2015,Seattle,WA,98121,NonResidential,Hotel,1980,119890,120.100000,122.100000,228.800000,227.100000,1.482910e+07,1.507824e+07,0.0,6.066245e+06,8.763105e+06
4,9,2015,Seattle,WA,98101,Specialty,Police Station,1999,97288,135.700000,146.900000,313.500000,321.600000,1.205198e+07,1.304526e+07,0.0,7.271004e+06,4.781283e+06
5,10,2015,Seattle,WA,98101,NonResidential,Hotel,1926,83008,76.900000,79.600000,149.500000,158.200000,6.252842e+06,6.477493e+06,0.0,2.679698e+06,3.573255e+06
6,11,2015,Seattle,WA,98101,NonResidential,Other - Entertainment/Public Assembly,1926,102761,62.500000,71.800000,152.200000,160.400000,6.426022e+06,7.380086e+06,2003108.0,4.108004e+06,3.150790e+05
7,12,2015,Seattle,WA,98104,NonResidential,Hotel,1904,163984,77.000000,82.900000,174.600000,186.800000,1.263374e+07,1.358902e+07,0.0,7.355649e+06,5.278400e+06
8,13,2015,Seattle,WA,98104,Residential,Multifamily Housing,1910,63712,81.700000,92.700000,178.100000,190.800000,4.587231e+06,5.203458e+06,1483476.0,2.370050e+06,7.338030e+05
9,15,2015,Seattle,WA,98101,NonResidential,Hotel,1969,153163,101.100000,107.200000,200.800000,205.200000,1.471985e+07,1.561593e+07,3623542.0,6.337531e+06,4.759044e+06


In [9]:
data_combined.to_csv(r'C:\Users\cjros\DIRECT\Capstone\Data\EnergyConsumption\Seattle\Seattle_Energy_Consumption_cleaned.csv', index_label='UID')

### Code to trim Seattle building energy data into for consistent buildings year to year

In [12]:
data_trimmed = data_combined[data_combined.groupby(['BuildingID'])['BuildingID'].transform('size') > 2]
data_trimmed.sort_values(by='BuildingID')
data_trimmed

,BuildingID,DataYear,City,State,ZipCode,BuildingType,PrimaryPropertyUse,YearBuilt,PropertyGFATotal,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu)


In [11]:
#data_combined.to_csv(r'C:\Users\cjros\DIRECT\Capstone\Data\EnergyConsumption\Seattle\Seattle_Energy_Consumption_trimmed.csv', index_label='RID')